# Paraphrase ça !

Remplace les mots d'une phrase par des synonymes

In [9]:
# A faire tourner une seule fois
!pip install spacy requests
!python -m spacy download fr_core_news_sm

In [5]:
import spacy
from spacy import displacy
import requests
from bs4 import BeautifulSoup
import random
import fr_core_news_sm

### Analyse de la phrase

In [6]:
# nlp = spacy.load("fr_core_news_sm")
nlp = fr_core_news_sm.load()
 
sentence = input()
doc = nlp(sentence)

# Text Preprocessing | Lemmatization
print("\n" + f"Token\t\tLemma\t\tStopword\tDEP\t\tPOS".format('Token', 'Lemma', 'Stopword'))
print("-"*70)
for token in doc:
    print(f"{str(token)}\t\t{token.lemma_}\t\t{token.is_stop}\t\t{token.dep_}\t\t{token.pos_}")

pos_tochange = ['PRON', 'VERB', 'NOUN','ADJ','ADV']    
sentence_dict = {idx:str(token) for (idx,token) in enumerate(doc)}
to_synos = {idx:str(token) for (idx,token) in enumerate(doc) if token.pos_ in pos_tochange}

Galaxie

Token		Lemma		Stopword	DEP		POS
----------------------------------------------------------------------
Galaxie		galaxie		False		ROOT		NOUN


### Web Scrapping
Extraction des synonymes depuis le site [Synonymo](http://www.synonymo.fr/)

In [7]:
def extract_synos(word):
    
    url = 'http://www.synonymo.fr/syno/' + str(word)
    get = requests.get(url)
    get.encoding = get.apparent_encoding
    html = get.text
    
    soup = BeautifulSoup(html, "html.parser")
                         # features='lxml')
    raw_text = soup.prettify()
    
    ul = soup.find('ul', {'class':'synos'})
    
    try : 
        first = ul.findAll('a')
        synos = [i.next for i in first]
        syno = random.choice(synos)
        return syno
    
    except AttributeError:
        return word  

### Construction de la phrase

In [8]:
synos = [extract_synos(x) for x in to_synos.values()]
synos_dict = {}
for idx, val in zip(to_synos.keys(), synos):
    d = {idx: val}
    synos_dict.update(d)
    
print("\n" + f"IDX\tWORD\t\tSYNONYM")
print("-"*34)

for i, w, s in zip(to_synos.keys(), to_synos.values(), synos):
    print(f"{i}\t{w}\t\t{s}")
    
paraphrase_dict = sentence_dict.copy()
for i in synos_dict.keys():
    paraphrase_dict[i] = synos_dict[i] 

new_sentence = " ".join(paraphrase_dict.values())
print('\n\n"{}" devient "{}"'.format(sentence, new_sentence))


IDX	WORD		SYNONYM
----------------------------------
0	Galaxie		nébuleuse


"Galaxie" devient "nébuleuse"
